In [1]:
import os
import glob
import gzip

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm

import shapely.wkt as wkt
from shapely.geometry import Point, LineString, box
from shapely.ops import nearest_points

import lxml.etree as ET
import pickle

import network_io as nio

# Define the path to the folder containing the network directories
base_path = '../../../data/pop_1pct_with_policies/output_networks_100_filtered/'
network_folders = glob.glob(os.path.join(base_path, 'network_d_*'))
districts = gpd.read_file("../../../data/visualisation/districts_paris.geojson")
base_output_links_no_policies = '../../../data/pop_1pct/output_1pct/'

# Abstract

To do: die relative Analyse pro District machen. Wie hat sich capacity, freeflow speed und car volume in dem District geändert, in dem Policy eingeführt wurde? Und wie hat es sich netzwerkweit geändert?

In [2]:
def create_policy_key(folder_name):
    # Extract the relevant part of the folder name
    parts = folder_name.split('_')[1:]  # Ignore the first part ('network')
    district_info = '_'.join(parts).replace('d_', '')
    districts = district_info.split('_')
    return f"policy introduced in Arrondissement(s) {', '.join(districts)}"
    
# Plotting function
def plot_policy_data(key, df):
    arrondissement_number = key.replace('policy_', '').replace('_', ' ')
    arrondissement_number_cleaned = arrondissement_number.replace(" ", "_")

    print(f"Policy: {key}")
    
    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:2154")
    gdf = gdf.to_crs(epsg=4326)

    x_min = gdf.total_bounds[0] + 0.05
    y_min = gdf.total_bounds[1] + 0.05
    x_max = gdf.total_bounds[2]
    y_max = gdf.total_bounds[3]
    bbox = box(x_min, y_min, x_max, y_max)
    
    # Filter the network to include only the data within the bounding box
    gdf = gdf[gdf.intersects(bbox)]
    
    # Set up the plot
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    gdf[gdf['vol_car'] == 0].plot(ax=ax, color='lightgrey', linewidth=0.2, label='Network', zorder = 1)

    # Plot links with activity_count using the Viridis color scheme and log scale
    gdf.plot(column='vol_car', cmap='coolwarm', linewidth=1.5, ax=ax, legend=True,
             norm=LogNorm(vmin=gdf['vol_car'].min() + 1, vmax=gdf['vol_car'].max()),
             legend_kwds={'label': "Car volume", 'orientation': "vertical", 'shrink': 0.5})
    
    # districts.plot(ax=ax, facecolor='None', edgecolor='black', linewidth=1, label = "Arrondissements", zorder=3)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    
    # Customize the plot
    plt.title(f"{arrondissement_number}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend()
    plt.savefig("results/network_policies_new/" + f"{arrondissement_number_cleaned}.png", dpi=300, bbox_inches='tight')
    # plt.show()
    
# Function to iterate over result_dic and perform the required operations
def analyze_geodataframes(result_dic):
    # Extract the base network data for comparison
    base_gdf = result_dic.get("base_network_no_policies")
    if base_gdf is not None:
        base_vol_car = base_gdf['vol_car'].sum()
        base_gdf_car = base_gdf[base_gdf['modes'].str.contains('car')]
        base_capacity_car = base_gdf_car['capacity'].sum() 
        # base_freespeed_car = base_gdf_car['freespeed'].sum()

    for policy, gdf in result_dic.items():
        print(f"Policy: {policy}")
        
        # Filter edges where 'modes' contains 'car'
        gdf_car = gdf[gdf['modes'].str.contains('car')]
        total_capacity_car = round(gdf_car['capacity'].sum())
        print(f"Total capacity of edges with 'car' mode: {total_capacity_car}")
        
        total_freespeed_car = round(gdf_car['freespeed'].sum())
        print(f"Total freespeed of edges with 'car' mode: {total_freespeed_car}")

        total_vol_car = gdf['vol_car'].sum()
        print(f"Total 'vol_car': {total_vol_car}")

        if policy != "base_network_no_policies" and base_gdf is not None:
            vol_car_increase = ((total_vol_car - base_vol_car) / base_vol_car) * 100
            capacity_car_increase = ((total_capacity_car - base_capacity_car) / base_capacity_car) * 100
            # freespeed_car_increase = ((total_freespeed_car - base_freespeed_car) / base_freespeed_car) * 100

            print(f"Percentage increase in 'vol_car': {vol_car_increase:.2f}%")
            print(f"Percentage increase in capacity (car edges): {capacity_car_increase:.2f}%")
            # print(f"Percentage increase in freespeed (car edges): {freespeed_car_increase:.2f}%")
        
        print()

In [3]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}
for folder in network_folders:
    policy_key = create_policy_key(os.path.basename(folder))
    gdf = nio.read_network_data(folder)
    if gdf is not None:
        result_dic[policy_key] = gdf
        
base_network_no_policies = nio.read_network_data(base_output_links_no_policies)
result_dic["base_network_no_policies"] = base_network_no_policies

In [4]:
base_network_no_policies

,link,from_node,to_node,length,freespeed,capacity,lanes,modes,vol_car,osm:relation:route_master,...,osm:way:id,osm:way:access,osm:way:oneway,osm:way:highway,osm:relation:route,osm:way:railway,osm:way:name,storageCapacityUsedInQsim,osm:way:tunnel,geometry
0,100315,24972409,24972408,16.181257,8.333333,480.0,1.0,"bus,car,car_passenger",53,NaN,...,4216830.0,NaN,yes,residential,bicycle,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (651473.304 6861511.110, 651457.123..."
1,100316,5904976363,24983651,14.860209,8.333333,480.0,1.0,"bus,car,car_passenger,pt",62,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (651477.818 6861578.659, 651475.677..."
2,100317,24983651,5904976363,14.860209,8.333333,960.0,2.0,"bus,car,car_passenger,pt",24,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (651475.677 6861563.954, 651477.818..."
3,100321,664205947,24972376,22.264540,8.333333,960.0,2.0,"car,car_passenger",62,NaN,...,4216834.0,NaN,yes,residential,NaN,NaN,Boulevard Saint-Germain,NaN,NaN,"LINESTRING (651565.078 6861531.470, 651559.387..."
4,100324,24972376,24972375,64.853276,8.333333,480.0,1.0,"bus,car,car_passenger",72,NaN,...,4216833.0,NaN,yes,residential,bicycle,NaN,Rue Dupuytren,NaN,NaN,"LINESTRING (651559.387 6861509.945, 651502.714..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31630,pt_IDFM:480531,pt_IDFM:480531,pt_IDFM:480531,20.000000,20.000000,7999.2,1.0,"artificial,stopFacilityLink,subway",0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (649659.000 6866255.005, 649659.000..."
31631,pt_IDFM:480541,pt_IDFM:480541,pt_IDFM:480541,20.000000,20.000000,7999.2,1.0,"artificial,stopFacilityLink,subway",0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (649661.000 6866250.005, 649661.000..."
31632,pt_IDFM:480541_pt_IDFM:480286,pt_IDFM:480541,pt_IDFM:480286,699.892849,12.000000,7999.2,1.0,"artificial,subway",0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.479841,NaN,"LINESTRING (649661.000 6866250.005, 649784.000..."
31633,pt_IDFM:480964,pt_IDFM:480964,pt_IDFM:480964,20.000000,20.000000,7999.2,1.0,"artificial,stopFacilityLink,tram",0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (646505.000 6860021.005, 646505.000..."


In [5]:
for key, df in result_dic.items():
    print(key)
    # plot_policy_data(key, df)

policy introduced in Arrondissement(s) 13
policy introduced in Arrondissement(s) 14
policy introduced in Arrondissement(s) 15
policy introduced in Arrondissement(s) 12
policy introduced in Arrondissement(s) 1
policy introduced in Arrondissement(s) 10
policy introduced in Arrondissement(s) 11
policy introduced in Arrondissement(s) 16
base_network_no_policies


In [6]:
analyze_geodataframes(result_dic)

Policy: policy introduced in Arrondissement(s) 13
Total capacity of edges with 'car' mode: 31839810
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1614235
Percentage increase in 'vol_car': -0.53%
Percentage increase in capacity (car edges): -2.18%

Policy: policy introduced in Arrondissement(s) 14
Total capacity of edges with 'car' mode: 31966820
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1622768
Percentage increase in 'vol_car': -0.01%
Percentage increase in capacity (car edges): -1.79%

Policy: policy introduced in Arrondissement(s) 15
Total capacity of edges with 'car' mode: 31848970
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1616852
Percentage increase in 'vol_car': -0.37%
Percentage increase in capacity (car edges): -2.16%

Policy: policy introduced in Arrondissement(s) 12
Total capacity of edges with 'car' mode: 31604490
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1611862
Percentage increase in

## Save the results_dic, so that we can use it for ML methods

In [9]:
# Save the dictionary
with open('results/results_pop_1pct_toy_example.pkl', 'wb') as f:
    pickle.dump(result_dic, f)